In [4]:
import pandas as pd
import numpy_financial as npf
import matplotlib.pyplot as plt

# === Cargar CSV con producción anual ===
df = pd.read_csv("produccion_anual_por_sitio.csv")

# === Parámetros base ===
capacidad_kw = 50000
opex_anual_usd = 15 * capacidad_kw
vida_util = 20
vida_inversor_base = 12
costo_inversor_usd_kw = 100
precio_spot = 50
capex_base = 850
fcr_base = 0.08
perdidas_base = 0.14

parametros = {
    "FCR": [0.06, 0.10],
    "CapEx PV": [capex_base * 0.8, capex_base * 1.2],
    "Precio spot": [40, 60],
    "Vida inversor": [10, 20],
    "Pérdidas": [0.12, 0.16],
}

colores = {
    "FCR": "darkorange",
    "CapEx PV": "teal",
    "Precio spot": "cornflowerblue",
    "Vida inversor": "mediumseagreen",
    "Pérdidas": "lightcoral"
}

# === Función para calcular LCOE y VAN ===
def calcular_lcoe(prod_kwh, fcr, capex, spot, vida_inv, perdidas):
    energia_util = prod_kwh * (1 - perdidas)
    ingresos_anuales = energia_util / 1000 * spot
    flujos = [ingresos_anuales - opex_anual_usd] * vida_util
    if vida_inv < vida_util:
        flujos[vida_inv - 1] -= costo_inversor_usd_kw * capacidad_kw
    van = npf.npv(rate=fcr, values=flujos)
    lcoe = (fcr * capex * capacidad_kw + opex_anual_usd * vida_util) / (energia_util * vida_util)
    return lcoe * 1000, van

tabla_resultados = []
resumen_final = []

for _, row in df.iterrows():
    sitio = row["Sitio"]
    prod_anual = row["Produccion_Anual_kWh"]
    lcoe_base, van_base = calcular_lcoe(prod_anual, fcr_base, capex_base, precio_spot, vida_inversor_base, perdidas_base)

    resumen_final.append({
        "Sitio": sitio,
        "Producción Anual (kWh)": round(prod_anual, 2),
        "LCOE base (USD/MWh)": round(lcoe_base, 2),
        "VAN base (USD)": round(van_base, 2)
    })

    print(f"\n📍 {sitio.upper()} — Producción: {round(prod_anual):,} kWh")
    print(f"   🔹 LCOE base: {lcoe_base:.2f} USD/MWh")
    print(f"   💰 VAN base:  ${van_base:,.2f}")

    resultados_sens_lcoe = {}
    resultados_sens_van = {}

    for param, (low, high) in parametros.items():
        if param == "FCR":
            lcoe_low, van_low = calcular_lcoe(prod_anual, low, capex_base, precio_spot, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual, high, capex_base, precio_spot, vida_inversor_base, perdidas_base)
        elif param == "CapEx PV":
            lcoe_low, van_low = calcular_lcoe(prod_anual, fcr_base, low, precio_spot, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual, fcr_base, high, precio_spot, vida_inversor_base, perdidas_base)
        elif param == "Precio spot":
            lcoe_low, van_low = calcular_lcoe(prod_anual, fcr_base, capex_base, low, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual, fcr_base, capex_base, high, vida_inversor_base, perdidas_base)
        elif param == "Vida inversor":
            lcoe_low, van_low = calcular_lcoe(prod_anual, fcr_base, capex_base, precio_spot, low, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual, fcr_base, capex_base, precio_spot, high, perdidas_base)
        elif param == "Pérdidas":
            lcoe_low, van_low = calcular_lcoe(prod_anual, fcr_base, capex_base, precio_spot, vida_inversor_base, low)
            lcoe_high, van_high = calcular_lcoe(prod_anual, fcr_base, capex_base, precio_spot, vida_inversor_base, high)

        resultados_sens_lcoe[param] = (lcoe_low, lcoe_high)
        resultados_sens_van[param] = (van_low, van_high)

        tabla_resultados.append({
            "Sitio": sitio,
            "Parámetro": param,
            "LCOE base": round(lcoe_base, 2),
            "LCOE low": round(lcoe_low, 2),
            "LCOE high": round(lcoe_high, 2),
            "VAN base (USD)": round(van_base, 2),
            "VAN low (USD)": round(van_low, 2),
            "VAN high (USD)": round(van_high, 2)
        })

    # === Guardar gráficos LCOE y VAN ===
    for metric, resultados, base, ylabel, filename_tag in [
        ("LCOE", resultados_sens_lcoe, lcoe_base, "LCOE (USD/MWh)", "lcoe"),
        ("VAN", resultados_sens_van, van_base, "VAN (USD)", "van")
    ]:
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = range(len(parametros))
        valores = [v for pair in resultados.values() for v in pair]
        margen = (max(valores) - min(valores)) * 0.05
        ax.set_xlim(min(valores) - margen, max(valores) + margen)

        for i, param in enumerate(parametros):
            low, high = resultados[param]
            ancho = max(high, low) - min(high, low)
            ax.barh(i, ancho if ancho > 0.01 else 0.01, left=min(low, high),
                    color=colores[param], edgecolor='black')
            ax.text(min(low, high) - 0.01, i + 0.15, f"{low:,.2f}", ha='right', fontsize=9)
            ax.text(max(low, high) + 0.01, i - 0.15, f"{high:,.2f}", ha='left', fontsize=9)

        ax.axvline(base, color='red', linestyle='--', label=f"{metric} base: {base:,.2f}")
        ax.set_yticks(list(y_pos))
        ax.set_yticklabels(parametros)
        ax.set_xlabel(ylabel)
        ax.set_title(f"Análisis de sensibilidad {metric} - {sitio}")
        ax.grid(axis='x', linestyle='--', alpha=0.5)
        ax.legend()
        plt.tight_layout()
        nombre_archivo = f"{sitio.lower()}_sensibilidad_{filename_tag}.png"
        plt.savefig(nombre_archivo)
        plt.close()
        print(f"🖼️ Imagen guardada: {nombre_archivo}")

# === Exportar tablas finales ===
pd.DataFrame(tabla_resultados).to_csv("tabla_sensibilidad_lcoe_y_van.csv", index=False)
pd.DataFrame(resumen_final).to_csv("resumen_lcoe_van.csv", index=False)

print("\n📄 Archivos generados:")
print("✅ tabla_sensibilidad_lcoe_y_van.csv")
print("✅ resumen_lcoe_van.csv")



📍 CALAMA — Producción: 100,129,672 kWh
   🔹 LCOE base: 10.68 USD/MWh
   💰 VAN base:  $35,557,487.41
🖼️ Imagen guardada: calama_sensibilidad_lcoe.png
🖼️ Imagen guardada: calama_sensibilidad_van.png

📍 SALVADOR — Producción: 90,934,980 kWh
   🔹 LCOE base: 11.76 USD/MWh
   💰 VAN base:  $31,365,123.70
🖼️ Imagen guardada: salvador_sensibilidad_lcoe.png
🖼️ Imagen guardada: salvador_sensibilidad_van.png

📍 VALLENAR — Producción: 96,289,997 kWh
   🔹 LCOE base: 11.11 USD/MWh
   💰 VAN base:  $33,806,769.30
🖼️ Imagen guardada: vallenar_sensibilidad_lcoe.png
🖼️ Imagen guardada: vallenar_sensibilidad_van.png

📄 Archivos generados:
✅ tabla_sensibilidad_lcoe_y_van.csv
✅ resumen_lcoe_van.csv
